In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
import datetime
import pytz
import re
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

In [2]:
# BigQuery 클라이언트 생성
client = bigquery.Client()

utc_now = datetime.datetime.now(pytz.utc)
kst = pytz.timezone('Asia/Seoul')
kst_now = utc_now.astimezone(kst)
print(kst_now)
yesterday = kst_now - datetime.timedelta(days=1)
date_str = yesterday.strftime("%Y-%m-%d")

# 데이터셋 및 테이블 설정
dataset_id = 'airbridge_lake'
table_id = 'app_2024'

# 특정 Event_Category에 해당하는 데이터를 쿼리
query = f"""
    SELECT Semantic_Event_Properties, Custom_Event_Properties
    FROM `{dataset_id}.{table_id}`
    WHERE Event_Date = '{date_str}'
"""

query_job = client.query(query)
results = query_job.result()
result_list = list(results)

2024-07-02 18:04:57.972018+09:00


key-value 쌍이 늘어날 수 있는 점을 동적으로 대응하기 위함

In [11]:
for col in ['Semantic_Event_Properties', 'Custom_Event_Properties']:
    for row in result_list:
        json_data = getattr(row, col)
        if json_data:
            keys.update(re.findall(r'"([^"]+)":', json_data))

# 동적 쿼리 생성을 위한 SELECT 절 구성
select_statements = [f"MAX(CASE WHEN key = '{key}' THEN value END) AS {key}" for key in keys]
select_clause = ",\n  ".join(select_statements)

In [13]:
select_clause

"MAX(CASE WHEN key = 'datetime' THEN value END) AS datetime,\n  MAX(CASE WHEN key = 'incentive_product_name' THEN value END) AS incentive_product_name,\n  MAX(CASE WHEN key = 'achievementID' THEN value END) AS achievementID,\n  MAX(CASE WHEN key = 'transactionID' THEN value END) AS transactionID,\n  MAX(CASE WHEN key = 'score' THEN value END) AS score,\n  MAX(CASE WHEN key = 'challenge_id' THEN value END) AS challenge_id,\n  MAX(CASE WHEN key = 'productID' THEN value END) AS productID,\n  MAX(CASE WHEN key = 'place' THEN value END) AS place,\n  MAX(CASE WHEN key = 'listID' THEN value END) AS listID,\n  MAX(CASE WHEN key = 'campaignName' THEN value END) AS campaignName,\n  MAX(CASE WHEN key = 'price' THEN value END) AS price,\n  MAX(CASE WHEN key = 'rate' THEN value END) AS rate,\n  MAX(CASE WHEN key = 'originalContributionMargin' THEN value END) AS originalContributionMargin,\n  MAX(CASE WHEN key = 'scheduleID' THEN value END) AS scheduleID,\n  MAX(CASE WHEN key = 'sharedChannel' THEN 

In [ ]:
dynamic_query = f"""
WITH json_table AS (
    SELECT 
        Event_Date, Event_Datetime, Airbridge_Device_ID_Type,Airbridge_Device_ID,User_ID,
        Country, Client_IP_Country_Code,Client_IP_Subdivision,Client_IP_City
        Channel, Campaign, Ad_Group, Ad_Creative, Is_Re_engagement,
        Event_Category, Event_Label, Event_Action, Event_Value,
        Semantic_Event_Properties, Custom_Event_Properties, 
    FROM 
        `{dataset_id}.{table_id}`
    WHERE
        Event_Date = '{date_str}'
)

,keys_extracted AS (
  SELECT
    *,
    REGEXP_EXTRACT_ALL(Semantic_Event_Properties, r'"([^"]+)":') AS semantic_keys,
    REGEXP_EXTRACT_ALL(Custom_Event_Properties, r'"([^"]+)":') AS custom_keys
  FROM
    json_table
)

,unnested_keys AS (
  SELECT
    Event_Date, Event_Datetime, Airbridge_Device_ID_Type,Airbridge_Device_ID,User_ID,
    Country, Client_IP_Country_Code,Client_IP_Subdivision,Client_IP_City
    Channel, Campaign, Ad_Group, Ad_Creative, Is_Re_engagement,
    Event_Category, Event_Label, Event_Action, Event_Value,
    key,
    JSON_EXTRACT_SCALAR(Semantic_Event_Properties, CONCAT('$.', key)) AS value
  FROM
    keys_extracted,
    UNNEST(semantic_keys) AS key
  UNION ALL
  SELECT
    Event_Date, Event_Datetime, Airbridge_Device_ID_Type,Airbridge_Device_ID,User_ID,
    Country, Client_IP_Country_Code,Client_IP_Subdivision,Client_IP_City
    Channel, Campaign, Ad_Group, Ad_Creative, Is_Re_engagement,
    Event_Category, Event_Label, Event_Action, Event_Value,
    key,
    JSON_EXTRACT_SCALAR(Custom_Event_Properties, CONCAT('$.', key)) AS value
  FROM
    keys_extracted,
    UNNEST(custom_keys) AS key
),
pivot_table AS (
  SELECT
    Event_Date, Event_Datetime, Airbridge_Device_ID_Type,Airbridge_Device_ID,User_ID,
    Country, Client_IP_Country_Code,Client_IP_Subdivision,Client_IP_City
    Channel, Campaign, Ad_Group, Ad_Creative, Is_Re_engagement,
    Event_Category, Event_Label, Event_Action, Event_Value,
    key,value
  FROM
    unnested_keys
),
grp AS (
  SELECT 
    Event_Date, Airbridge_Device_ID, Event_Category, Event_Label, Event_Action, key, value, 
    SUM(Event_Value) AS Event_Value, COUNT(*) AS cnt, 
    ARRAY_AGG(Event_Datetime) AS Event_Datetimes,
    ARRAY_AGG(User_ID) AS User_IDs,
    MIN(Airbridge_Device_ID_Type) AS Airbridge_Device_ID_Type, 
    MIN(Country) AS Country, MIN(Client_IP_Country_Code) AS Client_IP_Country_Code,
    MIN(Client_IP_Subdivision) AS Client_IP_Subdivision, MIN(Client_IP_City) AS Client_IP_City, 
    MIN(Channel) AS Channel, MIN(Campaign) AS Campaign, MIN(Ad_Group) AS Ad_Group, 
    MIN(Ad_Creative) AS Ad_Creative, MIN(Is_Re_engagement) AS Is_Re_engagement
  FROM 
    pivot_table
  GROUP BY 
    Event_Date, Airbridge_Device_ID, Event_Category, Event_Label, Event_Action, key, value
),
total_grp AS (
  SELECT 
    Event_Date, Airbridge_Device_ID, Event_Category, 
    SUM(Event_Value) AS Event_Value, COUNT(*) AS Count,
    ARRAY_AGG(
      STRUCT(
        Event_Action AS Action, Event_Label AS Label, cnt AS count, Event_Value AS Value, key, value, Event_Datetimes, User_IDs)) AS Detail
  FROM 
    grp
  GROUP BY 
    Event_Date, Airbridge_Device_ID, Event_Category
)
SELECT * FROM total_grp;

"""


In [ ]:

# 동적 쿼리 실행
dynamic_query_job = client.query(dynamic_query)
dynamic_results = dynamic_query_job.result()

# 결과 출력
for row in dynamic_results:
    print(row)